<a href="https://colab.research.google.com/github/Nomaqui/ENTREGAS/blob/main/99_2_modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

Cargar y preprocesar datos de train

In [ ]:
df = pd.read_csv('train.csv')
y = df['RENDIMIENTO_GLOBAL']
X = df.drop(columns=['RENDIMIENTO_GLOBAL', 'ID', 'ESTU_PRIVADO_LIBERTAD'])

cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

imp_num = SimpleImputer(strategy='mean')
X[num_cols] = imp_num.fit_transform(X[num_cols])

imp_cat = SimpleImputer(strategy='most_frequent')
X[cat_cols] = imp_cat.fit_transform(X[cat_cols])

#sin agrupar los "poco frecuentes"

X_convertida = pd.get_dummies(X, columns=cat_cols, drop_first=True)

División y escalamiento

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X_convertida, y,  test_size=0.05, random_state=19, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Definición de modelos

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Modelo básico
clf = LogisticRegression(max_iter=200, solver='lbfgs', random_state=19)


Entrenamiento y evaluación

In [ ]:
clf.fit(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)

# Evaluación
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

Predicción en test.csv

In [ ]:
# Cargar test real
df_test = pd.read_csv("test.csv")
ids_test = df_test["ID"]
X_test_real = df_test.drop(columns=["ID"])

# Imputar y codificar test como se hizo con entrenamiento
X_test_real[num_cols] = imp_num.transform(X_test_real[num_cols])
X_test_real[cat_cols] = imp_cat.transform(X_test_real[cat_cols])
frecuencias = X['ESTU_PRGM_ACADEMICO'].value_counts()
poco_frecuentes = frecuencias[frecuencias <= 690].index
X_test_real['ESTU_PRGM_ACADEMICO'] = X_test_real['ESTU_PRGM_ACADEMICO'].replace(poco_frecuentes, 'POCO_FRECUENTE')
X_test_real = pd.get_dummies(X_test_real, columns=cat_cols, drop_first=True)

# Alinear columnas con entrenamiento
X_test_real = X_test_real.reindex(columns=X_convertida.columns, fill_value=0)

# Escalar
X_test_real_scaled = scaler.transform(X_test_real)

# Predecir
y_test_pred = eclf.predict(X_test_real_scaled)

# Crear archivo de envío
submission = pd.DataFrame({
    "ID": ids_test,
    "RENDIMIENTO_GLOBAL": y_test_pred
})

submission.to_csv("submission.csv", index=False)
print("✅ Archivo 'submission.csv' creado correctamente.")